In [1]:
!pip install -q transformers

In [2]:
import torch
import sklearn
import numpy as np
import pandas as pd
from torch import nn
from sklearn import metrics
from tqdm.notebook import tqdm
from transformers import AdamW
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.utils.class_weight import compute_class_weight

In [3]:
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
dev

device(type='cpu')

In [4]:
class PatDataset(Dataset):
    def __init__(self, px, dx, rx, y):
        self.px = px
        self.dx = dx
        self.rx = rx
        self.y = y
            
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.px[idx], self.dx[idx], self.rx[idx], self.y[idx]


class MyModel(nn.Module):
    def __init__(self, path=None):
        super().__init__()
        self.px_seq = nn.Sequential(
            nn.Linear(768, 128),
            nn.ReLU(),
            nn.LayerNorm(128, eps=1e-12, elementwise_affine=True)
            )
        self.dx_seq = nn.Sequential(
            nn.Linear(768, 128),
            nn.ReLU(),
            nn.LayerNorm(128, eps=1e-12, elementwise_affine=True)
            )
        self.rx_seq = nn.Sequential(
            nn.Linear(806, 128),
            nn.ReLU(),
            nn.LayerNorm(128, eps=1e-12, elementwise_affine=True)
            )
        self.concat_seq = nn.Sequential(
            nn.Linear(384, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2),  
            )

    def forward(self, px_embed, dx_embed, rx_embed):
        px_features = self.px_seq(px_embed)
        dx_features = self.px_seq(dx_embed)
        rx_features = self.rx_seq(rx_embed)
        concat = torch.cat((px_features, dx_features, rx_features), dim=1)
        return self.concat_seq(concat)


class Trainer:
    def __init__(self, train_data, val_data):
        self.train_data = train_data
        self.val_data = val_data
        if torch.cuda.is_available():
            self.dev = torch.device('cuda')
        else:
            self.dev = torch.device('cpu')
        print("Using: ", self.dev)

    @staticmethod
    def _roc_auc(y_true, y_raw_logits, class_idx=1):
        y_score = F.softmax(y_raw_logits, dim=1).cpu().detach().numpy()[:, class_idx]
        return metrics.roc_auc_score(y_true.cpu().detach().numpy(), y_score)

    @staticmethod
    def _accuracy(true, pred_proba, class_idx=None):
        preds = torch.argmax(pred_proba, dim=1)
        if class_idx is None:
            score = (true == preds).float().mean()
        else:
            score = (true==preds)[true==class_idx].float().mean()
        return score

    @staticmethod
    def _get_optimizer_with_decay(model, lr):
        no_decay = ['bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
             'weight_decay': 0.01, 'lr':lr},
            {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
             'weight_decay': 0.0, 'lr':lr}
             ]
        return AdamW(optimizer_grouped_parameters, lr=lr)

    def _get_loss_func(self, class_wts=None):
        if class_wts is not None:
            loss_fn = torch.nn.CrossEntropyLoss(weight=torch.tensor(class_wts).type(torch.float).to(self.dev))
        else:
            loss_fn = torch.nn.CrossEntropyLoss()
        return loss_fn
    
    @staticmethod
    def get_class_wts(y):
        return compute_class_weight('balanced', np.unique(y), y)

    def train_model(self, model, n_epochs=1, lr=1e-5, class_wts=None,
                    path_to_save_model='./model.tar', logging_step=5,
                    patience=3):
        loss_fn = self._get_loss_func(class_wts=class_wts)
        optimizer = Trainer._get_optimizer_with_decay(model, lr)

        model.to(self.dev)
        trn_loss = []
        val_loss = []
        val_accu = []
        max_val_accu = 0
        count = 0
        for epoch in range(n_epochs):
            print(f"Epoch: {epoch}")
            trn_loss_per_epoch = []
            model.train()
            for i, batch in enumerate(tqdm(self.train_data)):
                batch = [x.to(self.dev) for x in batch]
                outputs = model(*batch[:-1])
                loss = loss_fn(outputs, batch[-1])
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                del batch
                torch.cuda.empty_cache()
                trn_loss_per_epoch.append(float(loss))
                if i % logging_step == 0:
                    print("train_loss: ", np.mean(trn_loss_per_epoch))
            trn_loss.append(np.mean(trn_loss_per_epoch))

            model.eval()
            valid_loss_per_epoch = []
            yhat_lst = []
            y_lst = []
            with torch.no_grad():
                for batch in tqdm(self.val_data):
                    batch = [x.to(self.dev) for x in batch]
                    outputs = model(*batch[:-1])
                    valid_loss_per_epoch.append(float(loss_fn(outputs, batch[-1])))
                    yhat_lst.append(outputs)
                    y_lst.append(batch[-1])   
                tmp = Trainer._roc_auc(torch.cat(y_lst, dim=0), torch.cat(yhat_lst, dim=0))
            val_loss.append(np.mean(valid_loss_per_epoch))
            val_accu.append(tmp)
            
            print(f"Training Loss for epoch {epoch}: ", trn_loss[-1])
            print("Validation Loss: ", val_loss[-1], "| Validation roc_auc: ", val_accu[-1])

            if val_accu[-1] > max_val_accu:
                count = 0
                max_val_accu = val_accu[-1]
                torch.save({
                            'epoch': epoch,
                            'model_state_dict': model.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'training_loss': trn_loss[-1],
                            'val_loss': val_loss[-1],
                            'val_accuracy': val_accu[-1]
                            }, path_to_save_model)
                print(f"Checkpoint saved!")
            else:
                count = count + 1

            if count > patience:
                print("Stopping early, restoring best checkpoint..")
                model_parameters = torch.load(path_to_save_model, map_location = self.dev)
                model.load_state_dict(model_parameters['model_state_dict'])
                optimizer.load_state_dict(model_parameters['optimizer_state_dict'])
                break 
            print("Max roc_auc till now: ", max_val_accu)
        return model.eval(), optimizer

In [5]:
px_trn = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/trn_px_journey.pkl")
dx_trn = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/trn_dx_journey.pkl")
rx_trn = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/imp_rx_trn.pkl")
y_trn = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/y_trn.pkl")
rx_trn.shape

(23868, 806)

In [6]:
px_val = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/val_px_journey.pkl")
dx_val = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/val_dx_journey.pkl")
rx_val = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/imp_rx_val.pkl")
y_val = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/y_val.pkl")
rx_val.shape

(4212, 806)

In [7]:
px_test = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/test_px_journey.pkl")
dx_test = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/test_dx_journey.pkl")
rx_test = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/imp_rx_test.pkl")
y_test = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/y_test.pkl")
rx_test.shape

(7020, 806)

In [8]:
trn_ds = PatDataset(px_trn, dx_trn, rx_trn.values.astype(np.float32), y_trn['switch_flag'].tolist())
trn_dl = torch.utils.data.DataLoader(trn_ds, batch_size=512, num_workers=0, pin_memory=False)

In [9]:
val_ds = PatDataset(px_val, dx_val, rx_val.values.astype(np.float32), y_val['switch_flag'].tolist())
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=512, num_workers=0, pin_memory=False)

In [10]:
test_ds = PatDataset(px_test, dx_test, rx_test.values.astype(np.float32), y_test['switch_flag'].tolist())
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=512, num_workers=0, pin_memory=False)

In [11]:
class_wts = compute_class_weight('balanced', np.unique(y_trn['switch_flag'].values), y_trn['switch_flag'].values)
class_wts

array([ 0.51734004, 14.9175    ])

In [12]:
trainer = Trainer(trn_dl, val_dl)

Using:  cpu


In [13]:
model = MyModel()

model, optimizer = trainer.train_model(
    model, n_epochs=200, lr=1e-5, class_wts=class_wts,
    path_to_save_model='/content/drive/MyDrive/ColabData/saved_models/PatientBERT/only_top/sepsis-readmission-only-top.tar',
    logging_step=46, patience=10
    )

Epoch: 0


train_loss:  0.6936256885528564
train_loss:  0.6909998870910482




Training Loss for epoch 0:  0.6909998870910482
Validation Loss:  0.686121490266588 | Validation roc_auc:  0.5982080482778205
Checkpoint saved!
Max roc_auc till now:  0.5982080482778205
Epoch: 1


train_loss:  0.684659481048584
train_loss:  0.6863901348824196




Training Loss for epoch 1:  0.6863901348824196
Validation Loss:  0.6826364133093092 | Validation roc_auc:  0.6201152939577814
Checkpoint saved!
Max roc_auc till now:  0.6201152939577814
Epoch: 2


...


Training Loss for epoch 66:  0.42851033870210037
Validation Loss:  0.6880112820201449 | Validation roc_auc:  0.6965755011663539
Max roc_auc till now:  0.6993785833372531
Epoch: 67


train_loss:  0.4732394516468048
train_loss:  0.4237874207344461




Training Loss for epoch 67:  0.4237874207344461
Validation Loss:  0.6927308572663201 | Validation roc_auc:  0.6960389260832981
Max roc_auc till now:  0.6993785833372531
Epoch: 68


train_loss:  0.46867990493774414
train_loss:  0.4191458891046808




Training Loss for epoch 68:  0.4191458891046808
Validation Loss:  0.6973021957609389 | Validation roc_auc:  0.6954396344320928
Stopping early, restoring best checkpoint..


In [14]:
model_parameters = torch.load(
    "/content/drive/MyDrive/ColabData/saved_models/PatientBERT/only_top/sepsis-readmission-only-top.tar",
    map_location = dev
    )
model.load_state_dict(model_parameters['model_state_dict'])
model = model.eval()

In [15]:
dl = trn_dl
yhat_lst = []
y_lst = []
with torch.no_grad():
    for batch in tqdm(dl):
        yhat = model(*batch[:-1])
        yhat_lst.append(yhat)
        y_lst += list(batch[-1].cpu().numpy())

print(Trainer._roc_auc(torch.tensor(y_lst), torch.cat(yhat_lst, dim=0)))


0.8742168805271372


In [16]:
dl = val_dl
yhat_lst = []
y_lst = []
with torch.no_grad():
    for batch in tqdm(dl):
        yhat = model(*batch[:-1])
        yhat_lst.append(yhat)
        y_lst += list(batch[-1].cpu().numpy())

print(Trainer._roc_auc(torch.tensor(y_lst), torch.cat(yhat_lst, dim=0)))


0.6993785833372531


In [17]:
dl = test_dl
yhat_lst = []
y_lst = []
with torch.no_grad():
    for batch in tqdm(dl):
        yhat = model(*batch[:-1])
        yhat_lst.append(yhat)
        y_lst += list(batch[-1].cpu().numpy())

print(Trainer._roc_auc(torch.tensor(y_lst), torch.cat(yhat_lst, dim=0)))


0.679872371783816
